# PCINet - Training

*Automatic mapping of high-risk urban areas for Aedes aegypti infestation based on building facade image analysis*

## Dataset

In [1]:
### Set dstpath to a folder formatted to Pytorch ImageFolder Dataset

import pandas as pd

rootpath = '/mnt/DADOS_PARIS_1/matheusp/csv_segunda_leva/'
df = pd.read_csv(rootpath+'new_dados_2.csv')
df = df[['dt_cadastro', 'foto', 'fachada']]
display(df)

import os, shutil
dstpath = '/mnt/DADOS_PARIS_1/laranjeira/Datasets/ICM_facade'
if not os.path.isdir(dstpath): os.mkdir(dstpath)


# ## Create dataset formatted for pytorch 
    
# for clas in range(5):
#     claspath = os.path.join(dstpath, str(clas))
#     if not os.path.isdir(claspath): os.mkdir(claspath)

# for k in range(len(df)):
#     print(f'\r{k}/{len(df)}', flush=True, end='')
#     row = df.iloc[k]
#     foto = row.foto
#     fotopath = os.path.join(rootpath, foto)
#     if os.path.isfile(fotopath):
#         clas = row.fachada
#         savepath = os.path.join(dstpath, str(clas-1), '_'.join(foto.split('/')[-2:]))
#         shutil.copyfile(fotopath, savepath)

,dt_cadastro,foto,fachada
0,2021-09-20,arquivos/20_09_2021/CEC7.tmp.jpg,5
1,2021-09-20,arquivos/20_09_2021/CFE2.tmp.jpg,5
2,2021-09-20,arquivos/20_09_2021/D0DE.tmp.jpg,5
3,2021-09-20,arquivos/20_09_2021/D10F.tmp.jpg,5
4,2021-09-20,arquivos/20_09_2021/D120.tmp.jpg,5
...,...,...,...
4166,2022-04-14,arquivos/19_04_2022/2FB5.tmp.jpg,3
4167,2022-04-28,arquivos/29_04_2022/614B.tmp.jpg,4
4168,2022-05-27,arquivos/24_06_2022/412B.tmp.jpg,2
4169,2022-05-24,arquivos/24_06_2022/6D08.tmp.jpg,4


In [2]:
from torch.utils.data import DataLoader, WeightedRandomSampler, random_split
from torchvision import datasets, transforms, models
import torchvision.transforms.functional as TF
from torch import nn, optim
import torch
import numpy as np

class idxSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, indices):
        self.indices = indices
        
    def __iter__(self):
        return iter(self.indices)

class MyRotationTransform:
    """Rotate by one of the given angles."""

    def __init__(self, angles):
        self.angles = angles

    def __call__(self, x):
        angle = np.random.choice(self.angles, p=[0.2, 0.6, 0.2])
        return TF.rotate(x, int(angle))

rotation_transform = MyRotationTransform(angles=[-90, 0, 90])


mean = [0.485, 0.456, 0.406] 
std = [0.229, 0.224, 0.225]

data_transforms =  transforms.Compose([
    transforms.Resize((384, 384)),
#     transforms.CenterCrop(384),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.1),
    rotation_transform,
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 3.)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

# data_transforms =  transforms.Compose([
#     transforms.Resize(384),
#     transforms.CenterCrop(384),
#     transforms.ToTensor(),
#     transforms.Normalize(mean, std),
# ])

dataset = datasets.ImageFolder(dstpath, transform=data_transforms)

## Metrics function

In [10]:
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, classification_report
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, top_k_accuracy_score
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

def calculate_metrics(preds, labels, score):
    preds, labels = np.asarray(preds), np.asarray(labels)
    score = np.asarray(score)
    
    cm = confusion_matrix(labels, preds, normalize='true')
    b_acc = balanced_accuracy_score(labels, preds)
    acc = accuracy_score(labels, preds)
    kappa = cohen_kappa_score(labels, preds)
    f1 = f1_score(labels, preds, average = 'weighted')  
    top2 = top_k_accuracy_score(labels, score, k=2)
    top3 = top_k_accuracy_score(labels, score, k=3)
    
    print ("\nAccuracy: " + str(acc))
    print ("Balanced_Accuracy: " + str(b_acc))
    print ("Kappa: " + str(kappa))
    print ("F1: " + str(f1))
    print ("Top-2: ", str(top2))
    print ("Top-3: ", str(top3))
    print (cm)
    print(classification_report(labels, preds))
    
    return b_acc

## Define focal loss

In [4]:
import torch.nn.functional as F
class FocalLoss(nn.CrossEntropyLoss):
    ''' Focal loss for classification tasks on imbalanced datasets '''

    def __init__(self, gamma, alpha=None, ignore_index=-100, reduction='none'):
        super().__init__(weight=alpha, ignore_index=ignore_index, reduction='none')
        self.reduction = reduction
        self.gamma = gamma

    def forward(self, input_, target):
        cross_entropy = super().forward(input_, target)
        # Temporarily mask out ignore index to '0' for valid gather-indices input.
        # This won't contribute final loss as the cross_entropy contribution
        # for these would be zero.
        target = target * (target != self.ignore_index).long()
        input_prob = torch.gather(F.softmax(input_, 1), 1, target.unsqueeze(1))
        loss = torch.pow(1 - input_prob, self.gamma) * cross_entropy
        return torch.mean(loss) if self.reduction == 'mean' \
               else torch.sum(loss) if self.reduction == 'sum' \
               else loss

## Train loop

In [ ]:
import pickle
num_folds = 5
with open('folds.dict', 'rb') as fp:
    folds = pickle.load(fp)
    
for fold, (train_ids, test_ids) in enumerate(zip(folds['train'], folds['test'])):
    
    # Print
    print('-'*25, f'FOLD {fold}', '-'*25)
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    train_loader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=16, sampler=train_subsampler)
    val_loader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=16, sampler=test_subsampler)
    
    
    model = models.efficientnet_v2_s(pretrained=True)
    num_ftrs = model.classifier[1].in_features
    model.classifier = nn.Sequential(
                                nn.Dropout(p=0.2, inplace=True),
                                nn.Linear(in_features=num_ftrs, out_features=5, bias=True))
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    gamma = 2
    class_weights = torch.FloatTensor([4.7, 1.0, 0.54, 0.5, 1.2]).to(device) # 4.5, 1.0, 0.5, 0.5, 1.5
    criterion = FocalLoss(gamma=gamma, alpha = class_weights, reduction='mean').to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=5e-5)
    
    epochs = 51
    best = {'epoch': 0, 'loss': 0.0, 'val_pred': None, 'val_score': None, 'val_labels': None}
    min_val_loss = -1

    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5, verbose=True)

    for ep in range(epochs):
        print('##############################################')
        print('Starting epoch ' + str(ep + 1) + '/' + str(epochs) + '...')

        # Setting model to training mode.
        print('    Training...', end='')
        model.train()
        losses, correct = [], 0.
        train_pred, train_score, train_labels = [], [], []
        for it, data in enumerate(train_loader):
            print(f'\r{it}/{len(train_loader)}', end='', flush=True)
            inps, labs = data

            inps = inps.to(device)
            labs = labs.to(device)

            optimizer.zero_grad()
            output = model(inps)
            _, preds = torch.max(output, 1)
            train_labels += labs.cpu().numpy().tolist()
            train_pred += output.max(1)[1].cpu().numpy().tolist()
            train_score += output.detach().cpu().numpy().tolist()
            correct += torch.sum(preds == labs.data)

            loss = criterion(output, labs)
            loss.backward() 
            optimizer.step()

            losses.append(loss.data.item())

        tr_loss = np.mean(losses)
        print(f'\nLoss: {np.mean(losses)}, Acc: {correct/len(train_ids)}', )
        print(np.histogram(train_labels, bins=5))
        calculate_metrics(train_pred, train_labels, train_score)
    
        print('    Testing...')
        model.eval()
        losses, correct = [], 0.
        val_pred, val_score, val_labels = [], [], []
        with torch.no_grad():
            for it, data in enumerate(val_loader):
                inps, labs = data

                inps = inps.to(device)
                labs = labs.to(device)
                output = model(inps)
                _, preds = torch.max(output, 1)
                val_labels += labs.cpu().numpy().tolist()
                val_pred += output.max(1)[1].cpu().numpy().tolist()
                val_score += output.detach().cpu().numpy().tolist()
                correct += torch.sum(preds == labs.data)

                loss = criterion(output, labs)
                losses.append(loss.data.item()) 

        print(f'Loss: {np.mean(losses)}, Acc: {correct/len(test_ids)}', )
        print(np.histogram(val_labels, bins=5))

        scheduler.step()

        b_acc = calculate_metrics(val_pred, val_labels, val_score)


        if b_acc > min_val_loss:
            print('='*15, 'BEST', '='*15)
            best['val_pred']   = val_pred
            best['val_score']  = val_score
            best['val_labels'] = val_labels
            best['loss']  = np.mean(losses)
            best['epoch'] = ep

            min_val_loss = b_acc
            torch.save(model.state_dict(), f'best_fold0{fold+1}.pt')

------------------------- FOLD 0 -------------------------


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Adjusting learning rate of group 0 to 1.0000e-05.
##############################################
Starting epoch 1/51...
202/203ining...
Loss: 1.0014268620848068, Acc: 0.25615763664245605
(array([ 113,  533, 1020, 1139,  443]), array([0. , 0.8, 1.6, 2.4, 3.2, 4. ]))

Accuracy: 0.2561576354679803
Balanced_Accuracy: 0.24427435152792984
Kappa: 0.03572354575797487
F1: 0.2435290504992442
Top-2:  0.479064039408867
Top-3:  0.6862684729064039
[[ 12  33   5  39  24]
 [ 28 184  36 185 100]
 [ 43 274  86 365 252]
 [ 27 315  77 403 317]
 [ 13 101  26 156 147]]
              precision    recall  f1-score   support

           0       0.10      0.11      0.10       113
           1       0.20      0.35      0.26       533
           2       0.37      0.08      0.14      1020
           3       0.35      0.35      0.35      1139
           4       0.17      0.33      0.23       443

    accuracy                           0.26      3248
   macro avg       0.24      0.24      0.22      3248
weighted avg

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##############################################
Starting epoch 4/51...
202/203ining...
Loss: 0.7724713176635686, Acc: 0.42980295419692993
(array([ 113,  533, 1020, 1139,  443]), array([0. , 0.8, 1.6, 2.4, 3.2, 4. ]))

Accuracy: 0.42980295566502463
Balanced_Accuracy: 0.3011114778837519
Kappa: 0.17511608829814063
F1: 0.3830987111627724
Top-2:  0.7318349753694581
Top-3:  0.9064039408866995
[[  2  64  21  26   0]
 [  2 225 163 142   1]
 [  9 134 371 505   1]
 [  2  79 257 797   4]
 [  2  17  74 349   1]]
              precision    recall  f1-score   support

           0       0.12      0.02      0.03       113
           1       0.43      0.42      0.43       533
           2       0.42      0.36      0.39      1020
           3       0.44      0.70      0.54      1139
           4       0.14      0.00      0.00       443

    accuracy                           0.43      3248
   macro avg       0.31      0.30      0.28      3248
weighted avg       0.38      0.43      0.38      3248

    Te

...

Loss: 0.5522399836895513, Acc: 0.46182265877723694
(array([ 36, 119, 246, 310, 101]), array([0. , 0.8, 1.6, 2.4, 3.2, 4. ]))
Adjusting learning rate of group 0 to 5.0000e-06.

Accuracy: 0.4618226600985222
Balanced_Accuracy: 0.42433095430879425
Kappa: 0.2666663911151157
F1: 0.462225721978417
Top-2:  0.7746305418719212
Top-3:  0.9273399014778325
[[  9  22   2   2   1]
 [  8  66  34   9   2]
 [  7  49 106  68  16]
 [  5  19  78 155  53]
 [  1   1  12  48  39]]
              precision    recall  f1-score   support

           0       0.30      0.25      0.27        36
           1       0.42      0.55      0.48       119
           2       0.46      0.43      0.44       246
           3       0.55      0.50      0.52       310
           4       0.35      0.39      0.37       101

    accuracy                           0.46       812
   macro avg       0.42      0.42      0.42       812
weighted avg       0.47      0.46      0.46       812

##############################################
St

##############################################
Starting epoch 19/51...
202/203ining...
Loss: 0.3837033897051083, Acc: 0.578201949596405
(array([ 108,  557, 1041, 1092,  450]), array([0. , 0.8, 1.6, 2.4, 3.2, 4. ]))

Accuracy: 0.5782019704433498
Balanced_Accuracy: 0.5743091099516888
Kappa: 0.42337534367711227
F1: 0.5771013452613484
Top-2:  0.8405172413793104
Top-3:  0.9593596059113301
[[ 70  26   9   1   2]
 [ 39 300 166  43   9]
 [ 20 135 599 251  36]
 [ 13  31 218 697 133]
 [  3   8  35 192 212]]
              precision    recall  f1-score   support

           0       0.48      0.65      0.55       108
           1       0.60      0.54      0.57       557
           2       0.58      0.58      0.58      1041
           3       0.59      0.64      0.61      1092
           4       0.54      0.47      0.50       450

    accuracy                           0.58      3248
   macro avg       0.56      0.57      0.56      3248
weighted avg       0.58      0.58      0.58      3248

    Test

Loss: 0.5527214708865857, Acc: 0.45443350076675415
(array([ 36, 119, 246, 310, 101]), array([0. , 0.8, 1.6, 2.4, 3.2, 4. ]))
Adjusting learning rate of group 0 to 2.5000e-06.

Accuracy: 0.4544334975369458
Balanced_Accuracy: 0.43538847730682795
Kappa: 0.26672224951534884
F1: 0.45799207372422723
Top-2:  0.7684729064039408
Top-3:  0.9334975369458128
[[ 11  18   3   2   2]
 [ 17  64  28   9   1]
 [ 12  48 109  57  20]
 [  3  17  85 141  64]
 [  1   1  13  42  44]]
              precision    recall  f1-score   support

           0       0.25      0.31      0.28        36
           1       0.43      0.54      0.48       119
           2       0.46      0.44      0.45       246
           3       0.56      0.45      0.50       310
           4       0.34      0.44      0.38       101

    accuracy                           0.45       812
   macro avg       0.41      0.44      0.42       812
weighted avg       0.47      0.45      0.46       812

##############################################

202/203ining...
Loss: 0.31545540455526905, Acc: 0.6105295419692993
(array([ 108,  557, 1041, 1092,  450]), array([0. , 0.8, 1.6, 2.4, 3.2, 4. ]))

Accuracy: 0.6105295566502463
Balanced_Accuracy: 0.6150795288069073
Kappa: 0.4674099843387902
F1: 0.6097816578066324
Top-2:  0.8713054187192119
Top-3:  0.9676724137931034
[[ 77  23   6   1   1]
 [ 36 314 165  33   9]
 [ 11 122 637 237  34]
 [ 10  23 214 716 129]
 [  2   3  39 167 239]]
              precision    recall  f1-score   support

           0       0.57      0.71      0.63       108
           1       0.65      0.56      0.60       557
           2       0.60      0.61      0.61      1041
           3       0.62      0.66      0.64      1092
           4       0.58      0.53      0.55       450

    accuracy                           0.61      3248
   macro avg       0.60      0.62      0.61      3248
weighted avg       0.61      0.61      0.61      3248

    Testing...
Loss: 0.5765229557074752, Acc: 0.46305418014526367
(array([ 36,

Loss: 0.5856631868610195, Acc: 0.4876847267150879
(array([ 36, 119, 246, 310, 101]), array([0. , 0.8, 1.6, 2.4, 3.2, 4. ]))
Adjusting learning rate of group 0 to 1.2500e-06.

Accuracy: 0.4876847290640394
Balanced_Accuracy: 0.4368405055634258
Kappa: 0.30387617028029024
F1: 0.48774424771601654
Top-2:  0.7931034482758621
Top-3:  0.9421182266009852
[[  5  19   6   2   4]
 [  5  68  36   9   1]
 [ 10  41 117  61  17]
 [  4  12  78 156  60]
 [  1   3  11  36  50]]
              precision    recall  f1-score   support

           0       0.20      0.14      0.16        36
           1       0.48      0.57      0.52       119
           2       0.47      0.48      0.47       246
           3       0.59      0.50      0.54       310
           4       0.38      0.50      0.43       101

    accuracy                           0.49       812
   macro avg       0.42      0.44      0.43       812
weighted avg       0.49      0.49      0.49       812

##############################################
S

## Test

In [16]:
from torchvision.transforms.functional import to_pil_image
from torchvision import datasets, transforms
from torchvision import models
from torch import nn
import torch, pickle
import numpy as np

with open('folds.dict', 'rb') as fp:
    folds = pickle.load(fp)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = models.efficientnet_v2_s(pretrained=False)
num_ftrs = model.classifier[1].in_features
model.classifier = nn.Sequential(
                            nn.Dropout(p=0.2, inplace=True),
                            nn.Linear(in_features=num_ftrs, out_features=5, bias=True))

    
mean = [0.485, 0.456, 0.406] 
std = [0.229, 0.224, 0.225]

data_transforms =  transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

dstpath = '/mnt/DADOS_PARIS_1/laranjeira/Datasets/ICM_facade'
dataset = datasets.ImageFolder(dstpath, transform=data_transforms)
    
results = {'filename': [],  'idx': [], 'label': [], 'pred': [], 'error': [], 'fold': []}
for fold, test_ids in enumerate(folds['test']):
    model.load_state_dict(torch.load(f'best_fold0{fold+1}.pt'))
    model.to(device)
    model.eval()
    
    error = []
    
    with torch.no_grad():
        
        val_pred, val_score, val_labels = [], [], [] 
        for k, idx in enumerate(test_ids):
            print(f'\r{k}/{len(test_ids)}', flush=True, end='')
            img, lab = dataset[idx]
            img = img.to(device)

            out = model(img[None])
            
            _, preds = torch.max(out, 1)
            val_labels += [lab]
            val_pred   += out.max(1)[1].cpu().numpy().tolist()
            val_score  += out.detach().cpu().numpy().tolist()

            pred = out.max(1)[1].cpu().numpy().tolist()
            error += [np.abs(lab - pred[0])]
            results['filename'].append(dataset.imgs[idx][0])
            results['idx'].append(idx)
            results['label'].append(dataset.imgs[idx][1])
            results['pred'].append(pred[0])
            results['error'].append(error[-1])
            results['fold'].append(fold)
            
        b_acc = calculate_metrics(val_pred, val_labels, val_score)
        
    print('')
        

811/812
Accuracy: 0.45197044334975367
Balanced_Accuracy: 0.42642153074578165
Kappa: 0.25416633985444115
F1: 0.4517296397481805
Top-2:  0.7413793103448276
Top-3:  0.9261083743842364
[[0.41935484 0.38709677 0.12903226 0.03225806 0.03225806]
 [0.11888112 0.37762238 0.38461538 0.11188811 0.00699301]
 [0.03745318 0.15355805 0.4906367  0.27715356 0.0411985 ]
 [0.03422053 0.03041825 0.28136882 0.50190114 0.15209125]
 [0.01851852 0.01851852 0.17592593 0.44444444 0.34259259]]
              precision    recall  f1-score   support

           0       0.25      0.42      0.32        31
           1       0.46      0.38      0.42       143
           2       0.46      0.49      0.48       267
           3       0.49      0.50      0.49       263
           4       0.41      0.34      0.37       108

    accuracy                           0.45       812
   macro avg       0.42      0.43      0.42       812
weighted avg       0.46      0.45      0.45       812


811/812
Accuracy: 0.45935960591133007


In [17]:
import pandas as pd

pd.DataFrame(results).to_csv('inferencias_5lcasses_data_augment.csv')